MultiAgent RAG 

In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap= 200).split_documents(docs)
vectordb = FAISS.from_documents(documents, OllamaEmbeddings(model="llama3.2"))
retriever = vectordb.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever, "langsmith_search", "Search for information about LangSmith. Use this tool for any queries on LangSmith")
retriever_tool.name

'langsmith_search'

In [5]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max = 200)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool.name

'arxiv'

In [6]:
tools = [wiki_tool,arxiv_tool,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/saipuneeth/Desktop/Projects/GenAI-/.venv/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. Use this tool for any queries on LangSmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x109298cc0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.ve

In [7]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_ollama import OllamaLLM, ChatOllama

llm = ChatOllama(model='llama3.2',temperature=0.6)

### Set up the environment variables
os.environ["LANGSMITH_API_KEY"]= os.getenv('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"]= 'true'
os.environ["LANGSMITH_ENDPOINT"]= os.getenv('LANGSMITH_ENDPOINT')

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 6


In [11]:
from langchain import hub

prompt = hub.pull(owner_repo_commit="sa2812/rag-prompt")
# prompt.messages


[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]

In [14]:
from langchain.agents import create_structured_chat_agent, create_openai_functions_agent
agent = create_openai_functions_agent(llm, tools, prompt=prompt)


ValueError: Prompt must have input variable `agent_scratchpad`, but wasn't found. Found ['context', 'question'] instead.